# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.power import TTestIndPower, TTestPower
power_analysis = TTestIndPower()
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')

In [2]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df.id.value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
604441    2
832343    2
344770    2
371595    2
604449    2
256844    2
751587    2
524996    2
525030    2
609072    2
624959    2
506185    2
637193    2
443177    2
632767    2
281261    2
761973    2
400039    2
434522    2
226772    2
674143    2
729442    2
766743    2
493927    2
260886    2
         ..
190376    1
196531    1
374714    1
664162    1
389065    1
344011    1
249813    1
440282    1
786395    1
643143    1
466807    1
530286    1
902988    1
872245    1
747320    1
798521    1
554810    1
474947    1
543045    1
276298    1
642893    1
309098    1
366414    1
229203    1
481110    1
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [4]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [6]:
df_exp = df.loc[df.group == 'experiment']
df_ctrl = df.loc[df.group == 'control']

In [7]:
click_list = list(df.id.loc[df.action == 'click'])
views_list = list(df.id.loc[df.action == 'view'])

outliers = []
for person in click_list:
    if person in views_list:
        continue
    else:
        outliers.append(person)

In [8]:
outliers

[]

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [9]:
df_exp = df.loc[df.group == 'experiment']
df_ctrl = df.loc[df.group == 'control']

In [10]:
exp_cl = len(df.loc[(df.group == 'experiment') & (df.action == 'click')])
exp_view = len(df.loc[(df.group == 'experiment') & (df.action == 'view')])
print(f'Clicks: {exp_cl}\nViews: {exp_view}\nExperiment click rate: {exp_cl/exp_view}')

Clicks: 928
Views: 2996
Experiment click rate: 0.3097463284379172


In [11]:
ctrl_cl = len(df.loc[(df.group == 'control') & (df.action == 'click')])
ctrl_view = len(df.loc[(df.group == 'control') & (df.action == 'view')])
print(f'Clicks: {ctrl_cl}\nViews: {ctrl_view}\nControl click rate: {ctrl_cl/ctrl_view}')

Clicks: 932
Views: 3332
Control click rate: 0.2797118847539016


Null Hypothesis: control and experiment are identical

Alternative Hypothesis: experiment mean is greater than control

alpha = 0.05


    
    

In [12]:
def welch_t(a, b):
    """ Calculate Welch's t statistic for two samples. """
    numerator = a.mean() - b.mean()
    denominator = np.sqrt(a.var(ddof=1)/len(a) + b.var(ddof=1)/b.size)
    return abs(numerator/denominator)

def welch_df(a, b):
    """ Calculate the effective degrees of freedom for two samples. """
    num = (a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)**2
    denom = ((a.var(ddof=1)**2) / (((a.size)**2) * (a.size-1))) + ((b.var(ddof=1)**2) / (((b.size)**2) * (b.size-1)))
    return num/denom

def p_value(a, b, two_sided=False):
    t = welch_t(a,b)
    df = welch_df(a,b)
    p = 1 - stats.t.cdf(t, df)
    if two_sided:
        p += p
    return p

In [22]:
df1 = df.copy()
df1 = df1[~((df1['id'].isin(click_list)) & (df1['action'].isin(['view'])))]

In [24]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6328 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    6328 non-null object
id           6328 non-null int64
group        6328 non-null object
action       6328 non-null object
dtypes: int64(1), object(3)
memory usage: 247.2+ KB


In [25]:
df1['success'] = 1

In [32]:
df1.loc[df1['action'] == 'view', 'success'] = 0

In [41]:
df1_exp = df1.loc[df1.group == 'experiment']
df1_ctrl = df1.loc[df1.group == 'control']

In [45]:
# formula = 'success ~ C(group)'
# lm = ols(formula, df1).fit()
# table = sm.stats.anova_lm(lm, typ=2)
# print(table)

In [44]:
p_value(df1_exp.success, df1_ctrl.success)

0.004466402814337078

.004 < .05  so we fail to reject the null hypothesis...so it's likely that the experimental version is significantly more effective than the control version

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [55]:
expected_clicks = len(df1_exp) * df1_ctrl.success.mean()
expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [58]:
n = len(df1_exp)
p = df1_ctrl.success.mean()
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [62]:
actual_clicks = sum(df1_exp.success)
z_score = (actual_clicks - expected_clicks) / std
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [63]:
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
looks good

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.